# 使用HTML文档分割器

加载文档后，您通常会想要对其进行转换以更好地适合您的应用程序。最简单的例子是，您可能希望将长文档分割成更小的块，以适合模型的上下文窗口。 LangChain 有许多内置的文档转换器，可以轻松地拆分、组合、过滤和以其他方式操作文档。

当您想要处理长文本时，有必要将该文本分割成块。这听起来很简单，但这里存在很多潜在的复杂性。理想情况下，您希望将语义相关的文本片段保留在一起。 “语义相关”的含义可能取决于文本的类型。本笔记本展示了实现此目的的几种方法。

在较高层面上，文本分割器的工作原理如下：
- 将文本分成小的、具有语义意义的块（通常是句子）。
- 开始将这些小块组合成一个更大的块，直到达到一定的大小（通过某些函数测量）。
- 一旦达到该大小，请将该块设为自己的文本片段，然后开始创建具有一些重叠的新文本块（以保持块之间的上下文）。

### HTMLHeaderTextSplitter

“MarkdownHeaderTextSplitter”、“HTMLHeaderTextSplitter”是一个“结构感知”分块器，它在元素级别拆分文本，并为每个与任何给定块“相关”的标题添加元数据。它可以逐个元素返回块，或者将元素与相同的元数据组合起来，目的是 (a) 保持相关文本在语义上（或多或少）分组，以及 (b) 保留文档结构中编码的上下文丰富的信息。它可以与其他文本分割器一起使用，作为分块管道的一部分。

! pip install -qU langchain-text-splitters

In [1]:
from langchain_community.document_loaders import TextLoader

file_path = "/Users/libing/kk_datasets/html/Animation-system.html"

loader = TextLoader(file_path)

docs = loader.load()

docs


[Document(metadata={'source': '/Users/libing/kk_datasets/html/Animation-system.html'}, page_content='<!DOCTYPE html>\n<html lang="zh">\n\t<head>\n\t\t<meta charset="utf-8" />\n\t\t<base href="../../../" />\n\t\t<script src="page.js"></script>\n\t\t<link type="text/css" rel="stylesheet" href="page.css" />\n\t</head>\n\t<body>\n\t\t<h1>动画系统（[name]）</h1>\n\n\t\t<h2>概述</h2>\n\n\t\t<p class="desc">\n\t\t\t在three.js动画系统中，您可以为模型的各种属性设置动画：\n\t\t\t[page:SkinnedMesh]（蒙皮和装配模型）的骨骼，morph targets（变形目标），\n\t\t\t不同的材料属性（颜色，不透明度，布尔运算），可见性和变换。动画属性可以淡入、淡出、交叉淡化和扭曲。\n\t\t\t在相同或不同物体上同时发生的动画的权重和时间比例的变化可以独立地进行。\n\t\t\t相同或不同物体的动画也可以同步发生。\n\t\t\t<br /><br />\n\t\t\t为了在一个同构系统中实现所有这一切，\n\t\t\tthree.js的动画系统[link:https://github.com/mrdoob/three.js/issues/6881 在2015年彻底改变]（注意过时的信息！），\n\t\t\t它现在有一个与Unity/虚幻4引擎类似的架构。此页面简要阐述了这个系统中的主要组件以及它们如何协同工作。\n\t\t</p>\n\n\t\t<h3>动画片段（Animation Clips）</h3>\n\n\t\t<p class="desc">\n\t\t\t如果您已成功导入3D动画对象（无论它是否有骨骼或变形目标或两者皆有都不要紧）——\n\t\t\t例如使用[link:https://github.com/KhronosGroup/glTF-Bl

In [2]:
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

In [4]:
from langchain_text_splitters import HTMLHeaderTextSplitter

splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

html_splitter = splitter.split_text(docs[0].page_content)

html_splitter

[Document(metadata={'Header 1': '动画系统（[name]）', 'Header 2': '概述'}, page_content='在three.js动画系统中，您可以为模型的各种属性设置动画： [page:SkinnedMesh]（蒙皮和装配模型）的骨骼，morph targets（变形目标）， 不同的材料属性（颜色，不透明度，布尔运算），可见性和变换。动画属性可以淡入、淡出、交叉淡化和扭曲。 在相同或不同物体上同时发生的动画的权重和时间比例的变化可以独立地进行。 相同或不同物体的动画也可以同步发生。 为了在一个同构系统中实现所有这一切， three.js的动画系统[link:https://github.com/mrdoob/three.js/issues/6881 在2015年彻底改变]（注意过时的信息！）， 它现在有一个与Unity/虚幻4引擎类似的架构。此页面简要阐述了这个系统中的主要组件以及它们如何协同工作。'),
 Document(metadata={'Header 1': '动画系统（[name]）', 'Header 2': '概述', 'Header 3': '动画片段（Animation Clips）'}, page_content='如果您已成功导入3D动画对象（无论它是否有骨骼或变形目标或两者皆有都不要紧）—— 例如使用[link:https://github.com/KhronosGroup/glTF-Blender-IO glTF Blender exporter]（glTF Blender导出器） 从Blender导出它并使用[page:GLTFLoader]将其加载到three.js场景中 —— 其中一个响应字段应该是一个名为“animations”的数组， 其中包含此模型的[page:AnimationClip AnimationClips]（请参阅下面可用的加载器列表）。 每个*AnimationClip*通常保存对象某个活动的数据。 举个例子，假如mesh是一个角色，可能有一个AnimationClip实现步行循环， 第二个AnimationClip实现跳跃，第三个AnimationClip实现闪避等等。'),
 Document(metadata={'Header 1': '动画系统（[nam

创建本地向量数据库

In [6]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embedding_model = "/Users/libing/kk_LLMs/bge-large-zh-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

vector_store = Chroma.from_documents(html_splitter, embedding=embeddings)

retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})

retriever.get_relevant_documents("请告诉我关于动画的一些知识")

Number of requested results 20 is greater than number of elements in index 14, updating n_results = 14


[Document(metadata={'Header 1': '动画系统（[name]）', 'Header 2': '概述', 'Header 3': '动画对象组（Animation Object Groups）'}, page_content='如果您希望一组对象接收共享的动画状态，则可以使用[page:AnimationObjectGroup]。')]

构建提示词

In [23]:
from langchain_core.prompts import ChatPromptTemplate

template = """你是一个关于js动画制作的专家，你的名字叫{name}
请根据以下文档回答问题：
要求：
1. 回答问题时，要换行输出，尽量做到每句一行。
2. 回答问题时，要使用markdown格式输出，尤其涉及一些代码展示的部分。
3. 回答问题时，语气可以生动活泼一些，可以加上一些emoji点缀答案。
4. 回答结束后，要加上礼貌用语对用户的提问表示感谢。

{context}

问题：{question}
"""

prompt = ChatPromptTemplate.from_template(template)


对接大模型

In [28]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("ZHIPUAI_API_KEY")
base_url = os.getenv("ZHIPUAI_API_BASE")

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.0, max_tokens=8192, model="glm-4v-flash")

创建链

In [29]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter


setup_and_retriever = RunnableParallel(
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "name": itemgetter("name"),
    }
)

chain = setup_and_retriever | prompt | llm | StrOutputParser()

for s in chain.stream({"name": "小明", "question": "请告诉我关于动画的一些特别详细的知识"}):
    print(s, end="", flush=True)


Number of requested results 20 is greater than number of elements in index 14, updating n_results = 14


# 动画系统的奥秘 🎬🧠

## 概述 💡

动画制作是一项神奇的技艺，它能让无生命的物体或角色变得栩栩如生。在JavaScript的世界里，我们可以通过`AnimationClip`来控制这些动画的生命周期和行为。这个`AnimationClip`就像是一个容器，里面装着各种动画属性的**关键帧轨道（Keyframe Tracks）**。

### 关键帧轨道（Keyframe Tracks）

想象一下，我们有一个角色对象，比如一个可爱的猫咪。猫咪的动作包括移动、转身、眨眼等。为了让猫咪动起来，我们需要为每一个动作创建一个关键帧轨道。

- **位置（Position Track）**: 跟踪猫咪在不同时间的坐标位置。
  
- **旋转（Rotation Track）**: 记录猫咪头部和尾巴的转动角度。
  
- **缩放（Scale Track）**: 控制猫咪身体的大小变化。
  
- **颜色（Color Track）**: 如果是变色龙，那么它的皮肤颜色也会随着时间改变。

当然，这只是冰山一角。实际上，动画还可以更加复杂，比如添加声音效果、光照变化等。

## Morph Targets（变形目标）

除了基本的运动外，动画还可以实现角色的表情变化。例如，当猫咪听到惊吓的声音时，它会从微笑变成害怕的表情。这时，我们就需要用到**Morph Targets**了。

变形目标就像是预设好的脸部表情模板。当我们播放动画时，不同的变形目标会在适当的时间被激活，从而产生丰富的表情变化。

总的来说，动画制作是一门深奥的艺术，涉及到很多技术和创意。希望我这次的解释能让你对这个领域有所了解！如果你还有其他问题，随时欢迎提问哦！

感谢您的提问！如果您有任何其他问题或需要进一步的帮助，请随时告诉我。祝您学习愉快！

In [30]:
for s in chain.stream({"name": "小明", "question": "请说说如何控制动画的播放和暂停"}):
    print(s, end="", flush=True)

Number of requested results 20 is greater than number of elements in index 14, updating n_results = 14


小明在这里为您解答！

🎬 控制动画的播放和暂停，主要依赖于`AnimationMixer`这个类。您可以使用它的`play()`方法来开始播放动画，或者使用`stop()`方法来停止动画。如果您想暂停动画，可以使用`pause()`方法，想要继续播放之前暂停的动画，就调用`resume()`方法即可。

```markdown
# 如何控制动画的播放和暂停？

- 使用 `AnimationMixer.play()` 开始播放动画。
- 使用 `AnimationMixer.stop()` 停止动画。
- 使用 `AnimationMixer.pause()` 暂停动画。
- 使用 `AnimationMixer.resume()` 继续播放之前暂停的动画。
```

希望这些信息能帮到您！有更多问题，随时找我哦~ 😊

感谢您的提问！